In [ ]:
import numpy as np
import pandas as pd
import multiprocessing

import scipy
from scipy import spatial
from scipy.spatial import cKDTree

import sklearn as sk
from sklearn import svm
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn import clone
from sklearn.externals.six.moves import xrange

import matplotlib.pyplot as plt
import bokeh
import bokeh.io
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

# init_notebook_mode()

import seaborn as sns

import re
import math
import copy

from collections import defaultdict
import csv
import itertools
import datetime 
from datetime import datetime
import time
import dateutil.parser
import pickle
import random

import gc
import zipfile
import sys, getopt
import os

from IPython.core.interactiveshell import InteractiveShell
from io import StringIO

import dask.dataframe as dd
#from chest import Chest

InteractiveShell.ast_node_interactivity = "all"
#InteractiveShell.ast_node_interactivity = "last"

# Magic function to make matplotlib inline
%matplotlib inline

%config InlineBackend.figure_formats = {'png', 'retina'}

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

import dask.dataframe as ddf
import dask.array as da

pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

In [ ]:
# 2018: SS18/FW18 ranges
# SS18 = pd.read_excel('RangeSS18.xlsx', skiprows = 5) # **PROBLEM**
SS18 = pd.read_excel('SS18_range_data.xlsx') 
FW18 = pd.read_excel('RangeFW18.xlsx', skiprows = 5)

In [ ]:
SS18 = SS18[['Article Number', 'Article format (all 6 digits)']]
FW18 = FW18[['Article Number', 'Article Number (6 digits)']]

In [ ]:
SS18_range = SS18['Article Number'].unique()
FW18_range = FW18['Article Number'].unique()

In [ ]:
len(SS18_range)
len(FW18_range)

len(set(SS18_range).intersection(set(FW18_range)))

len(set(FW18_range).intersection(set(SS20_range)))

In [ ]:
# SS20 = (pd.read_excel('SS20Range.xlsx', skiprows = 6).
#        drop(['Unnamed: 0', 'vlookup'], axis = 1))
# SS20 = SS20[SS20['CO'] == 'Y']
# dat['CO'].fillna('N', inplace = True)
# dat['SS19 Ranged'].fillna('N', inplace = True)
# dat['FW19 Ranged'].fillna('N', inplace = True)

# dat.to_csv('dat_SS20_range.csv')
dat_SS20_range = pd.read_csv('dat_SS20_range.csv')
SS20_range = dat_SS20_range['Article Number'].unique()

In [ ]:
# EDA
dat_SS20_range = dat_SS20_range[['Article Number', 'Model', 'Product Division', 
                                 'Category Marketing Line', 'Gender', 'Model Number', 'Product Group',
                                 'Product Type', 'Sports Category', 'Key Category', 'WE eCom', 'SS19 Ranged', 
                                 'FW19 Ranged', 'CO']]


In [ ]:
dat_SS20_range.head()

In [ ]:
dat_SS20_range.groupby(['SS19 Ranged', 'FW19 Ranged', 'CO'])['Article Number'].count()
# dat_SS20_range['Key Category'].value_counts()[:10]
# dat_SS20_range['Product Division'].value_counts()
# dat_SS20_range['Product Group'].value_counts()[:10]
# dat_SS20_range['Gender'].value_counts()

# dat_SS20_range[dat_SS20_range['FW19 Ranged'] == 'Y'].groupby('Article Number')[]

## Corrections

In [ ]:
# *** Transactions: 2019 + 2018_Q4 ***** ---- ISSUES ---- ****

# dat = pd.read_csv('Transaction data full 2019.csv', low_memory=False, index_col = 0) 
# # Incomplete: dat2 = pd.read_csv('Transaction Data/Transaction Data Q4 2018.csv', low_memory=False, index_col = 0)

# # From David w/ Joerian's query
# dat2 = pd.read_csv('Transaction data Q42018.csv', low_memory=False, index_col = 0)

# dat = pd.concat([dat, dat2])
# del dat2
# dat = dat[dat['season'] == 'SS19']

# --------------------------------------------------------------------
# David exasol
# dat = pd.read_csv('SS2019.csv', low_memory=False, 
#                   names = ['order_date', 'articleno', 'quantity', 'order_item_paidprice_gross', 
#                            'order_item_listprice_gross', 'net_discount_value', 'storecountry_code', 
#                            'DDISCOUNTPERCENTAGE', 'promotioncodedescription', 'ssalesbrand'])



# # ----- Benoit's magic ----
# dat.loc[dat.order_date.apply(lambda x: "mkt-Raiffeisen Bank" in x),:].index

In [ ]:
dat = pd.read_csv('SS2019.csv', 
            low_memory=False, 
            skiprows= [109853, 109854, 109856, 119470, 212394, 212395, 212396, 212397,
            420307, 420308, 568303, 755985, 758593, 758598, 984903, 984906,
            984907, 984909, 984910],
            names = ['order_date', 'articleno', 'quantity', 'order_item_paidprice_gross', 
                     'order_item_listprice_gross', 'net_discount_value', 'storecountry_code', 
                     'DDISCOUNTPERCENTAGE', 'promotioncodedescription', 'ssalesbrand'])

dat.drop(['order_item_paidprice_gross', 'order_item_listprice_gross', 'order_item_paidprice_gross', 
          'net_discount_value', 'storecountry_code', 'DDISCOUNTPERCENTAGE', 'promotioncodedescription', 
          'ssalesbrand'], inplace = True, axis = 1)
dat.rename(columns = {'articleno': 'article_number', 
                      'quantity': 'gross_qty',
                      'order_date': 'consumer_order_date'}, 
           inplace = True)

dat['consumer_order_date'] = pd.to_datetime(dat['consumer_order_date'])

In [ ]:
dat.groupby('article_number')['gross_qty'].sum().describe()

In [ ]:
# Wall time: 26.2 s

# dat.reset_index(inplace = True)
# dat['consumer_order_date'] = pd.to_datetime(dat['consumer_order_date'])
# dat.drop(['country', 'brand', 'gross_sales_gross_disc_net_ret', 'net_sales', 'total_markdown', 'article_promotion_main_category_group', 'fraction_of_full_price', 'markdown', 'fw_or_ss'], inplace = True, axis = 1)

SS20_range = dat_SS20_range['Article Number'].unique()
dat = dat[[(a in SS20_range) for a in dat['article_number']]]

# 'aggregate' to weekly sums by article for buy_availability adjustment
dat.set_index('consumer_order_date', inplace = True)
dat = dat[['article_number', 'gross_qty']].groupby(['article_number']).resample('W').sum()
dat.reset_index(inplace=True)

# Add 'week' and 'year' for merging with stock (buy_availability) data (b/c min_date_of_week)
dat['week'] = [t.week for t in dat['consumer_order_date']]
dat['year'] = [t.year for t in dat['consumer_order_date']]

In [ ]:
dat.groupby('article_number')['gross_qty'].sum().describe()

In [ ]:
# Wall time: ~40 s

# Dec 2018 data
dat_stock = pd.read_csv('Stock Data/Stock data 2018.csv', low_memory=False, index_col = 0)
dat_stock['min_date_of_week'] = pd.to_datetime(dat_stock['min_date_of_week'])
dat_stock = dat_stock[dat_stock['min_date_of_week'] > pd.to_datetime('2018-12-1')] # filter to > Dec1 (SS19)

# 2019 data
dat_stock2 = pd.read_csv('Stock Data/Stock data 2019.csv', low_memory=False, index_col = 0)

# Build: Dec2018 + 2019
dat_stock = pd.concat([dat_stock, dat_stock2])
del dat_stock2

# Tidy
dat_stock.reset_index(inplace = True)
dat_stock.drop(['avg(ecom_available_stock)', 'avg(size_availability)'], axis = 1, inplace = True)
dat_stock.rename(columns = {'min_date_of_week': 'date', 'avg(buy_availability)': 'buy_availability'}, inplace = True)

# Filter to SS20_range carryovers
dat_stock = dat_stock[[(a in SS20_range) for a in dat_stock['article_number']]]
dat_stock['date'] = pd.to_datetime(dat_stock['date'])

# For merging with transaction data
dat_stock['week'] = [t.week for t in dat_stock['date']]
dat_stock['year'] = [t.year for t in dat_stock['date']]

In [ ]:
# transactions + buy_availability
dat = pd.merge(dat, dat_stock, 
               left_on = ['article_number', 'year', 'week'], 
               right_on = ['article_number', 'year', 'week'],
               how = 'outer')

dat['buy_availability'] = dat['buy_availability'].fillna(1)
dat.dropna(inplace = True) # NA rows are 'gross_qty = 0'

In [ ]:
# ----- Calculate observed gross_qty per season -----
dat_season = pd.DataFrame(dat.groupby(['article_number'])['gross_qty'].sum())
dat_season.rename(columns = {'gross_qty':'gross_qty_season'}, inplace = True)

In [ ]:
# WEEKLY averages for articles when fully stocked
dat_stocked = pd.DataFrame(
    dat[dat['buy_availability'] > 0.35].
    groupby(['article_number'])['gross_qty'].
    mean())

dat_stocked.rename(columns = {'gross_qty':'gross_qty_stocked_weekly_avg'}, inplace= True)

dat_stocked['stocked_season_projection'] = 26*dat_stocked['gross_qty_stocked_weekly_avg']

In [ ]:
dat_season = (pd.merge(dat_season, dat_stocked, left_index=True, right_index=True, how = 'outer').round())

dat_season.drop('gross_qty_stocked_weekly_avg', inplace=True, axis = 1)
dat_season.rename(columns = {'stocked_season_projection':'understock_correction'}, inplace= True)

In [ ]:
# --- Clearance correction ----

# --- *** NEED TO USE THE FULL SS19 DATA HERE TOO *** ---

dat_wo_clearance = pd.read_csv('Transaction data full 2019.csv', low_memory=False, index_col = 0) 
dat_wo_clearance2 = pd.read_csv('Transaction Data/Transaction Data Q4 2018.csv', low_memory=False, index_col = 0)
dat_wo_clearance = pd.concat([dat_wo_clearance, dat_wo_clearance2])
del dat_wo_clearance2

dat_wo_clearance = dat_wo_clearance[dat_wo_clearance['season'] == 'SS19']

dat_wo_clearance.reset_index(inplace = True)
dat_wo_clearance['consumer_order_date'] = pd.to_datetime(dat_wo_clearance['consumer_order_date'])
dat_wo_clearance.drop(
    ['country', 'brand', 'gross_sales_gross_disc_net_ret', 'net_sales', 'total_markdown',
     'article_promotion_main_category_group', 'fraction_of_full_price', 'markdown', 'fw_or_ss'],
    inplace = True, axis = 1)

In [ ]:
dat_wo_clearance = dat_wo_clearance[[(a in SS20_range) for a in dat_wo_clearance['article_number']]]

dat_wo_clearance = dat_wo_clearance[dat_wo_clearance['clearance'] == 0]

dat_wo_clearance = pd.DataFrame(dat_wo_clearance.groupby(['article_number', 'season'])['gross_qty'].sum())
dat_wo_clearance.rename(columns = {'gross_qty': 'overstock_correction'}, inplace = True)

In [ ]:
dat_season = pd.merge(dat_season, dat_wo_clearance, how = 'outer', left_index = True, right_index = True)
del dat_wo_clearance

In [ ]:
preds = (pd.DataFrame(
    pd.read_csv('Buyers predictions.csv', low_memory=False, index_col = 0))
         [['season', 'ecom_marketing_forecast']].
         reset_index().
        dropna())

preds = preds[preds['season'] == 'SS19']

In [ ]:
dat_season = pd.merge(
    dat_season, preds, 
    left_on = ['article_number'],
    right_on=['article'], 
    how = 'left').round()

# dat_season.drop('article', axis = 1, inplace = True)
del preds

In [ ]:
def projection(net, forecast, under, over):
    if net > forecast:
        return under
    elif net < forecast:
        return over
    else:
        return under

dat_season['gross_qty*'] = (dat_season.
                          apply(lambda row: projection(row['gross_qty_season'], 
                                                       row['ecom_marketing_forecast'], 
                                                       row['understock_correction'], 
                                                       row['overstock_correction']), axis = 1))
dat_season = dat_season[dat_season['gross_qty*'] != 0]

In [ ]:
dat_season['gross_qty_season'].describe()

In [ ]:
dat_season[['article', 'gross_qty_season', 'ecom_marketing_forecast']]